In [2]:
from pathlib import Path

# Basis-Verzeichnis: dort, wo dein Notebook liegt
BASE_DIR = Path().resolve()  # z.B. .../Erdbeeren_yolo/Riseholme/Riseholme-2021

# Relativer Pfad zu den ripe-Bildern
image_dir = BASE_DIR / 'Data' / 'Normal' / 'Ripe'
# Relativer Pfad, wo die Labels hinkommen sollen
label_dir = BASE_DIR / 'labels' / 'Ripe'

# Existenzauschecken und Ordner anlegen
if not image_dir.is_dir():
    raise FileNotFoundError(f"{image_dir} existiert nicht")
label_dir.mkdir(parents=True, exist_ok=True)

# Klasse 0 = "ripe"
class_id = 0
count = 0

# Jedes Bild → eine YOLO-Label mit Full-Image-Box
for img_path in image_dir.iterdir():
    if img_path.suffix.lower() in {'.jpg', '.jpeg', '.png'}:
        txt_path = label_dir / f"{img_path.stem}.txt"
        with open(txt_path, 'w') as f:
            # x_center=0.5, y_center=0.5, width=1.0, height=1.0
            f.write(f"{class_id} 0.5 0.5 1.0 1.0\n")
        count += 1

print(f"✅ {count} Labels erstellt in {label_dir}")


✅ 462 Labels erstellt in /home/parallels/Documents/Forschsem/Erdbeeren_yolo/Riseholme/Riseholme-2021/labels/Ripe


datensatz splitten

In [5]:
from pathlib import Path
from sklearn.model_selection import train_test_split
import shutil

BASE_DIR    = Path().resolve()                           # …/Riseholme-2021
IMG_SRC     = BASE_DIR / 'Data' / 'Normal' / 'Ripe'      # deine Original-Bilder
LBL_SRC     = BASE_DIR / 'labels' / 'Ripe'               # deine erzeugten .txt

# Zielordner für YOLO
IMG_TRAIN   = BASE_DIR / 'images' / 'train'
IMG_VAL     = BASE_DIR / 'images' / 'val'
LBL_TRAIN   = BASE_DIR / 'labels' / 'train'
LBL_VAL     = BASE_DIR / 'labels' / 'val'

# Ordner anlegen
for d in (IMG_TRAIN, IMG_VAL, LBL_TRAIN, LBL_VAL):
    d.mkdir(parents=True, exist_ok=True)

# Dateinamen (Stem) sammeln und splitten
files = [f.stem for f in IMG_SRC.iterdir() if f.suffix.lower() in {'.png','.jpg'}]
train, val = train_test_split(files, test_size=0.2, random_state=42)

# Kopieren
for f in train:
    shutil.copy(IMG_SRC/ f"{f}.png", IMG_TRAIN/ f"{f}.png")
    shutil.copy(LBL_SRC/ f"{f}.txt", LBL_TRAIN/ f"{f}.txt")
for f in val:
    shutil.copy(IMG_SRC/ f"{f}.png", IMG_VAL/ f"{f}.png")
    shutil.copy(LBL_SRC/ f"{f}.txt", LBL_VAL/ f"{f}.txt")

print(f"Train: {len(train)} | Val: {len(val)}")


Train: 369 | Val: 93


In [2]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')             # oder yolov8s.pt, -m.pt etc.
model.train(
    data='data.yaml',                  # deine Konfig
    epochs=50,                         # je nach Bedarf
    imgsz=640,
    batch=16,
    augment=True,
    project='runs/train',              # Ausgabeordner
    name='ripe_only'                   # Lauf-Name
)


Ultralytics 8.3.116 🚀 Python-3.13.3 torch-2.5.1 CPU (Intel Core(TM) i5-1038NG7 2.00GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=runs/train, name=ripe_only2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=No

train: Scanning /home/parallels/Documents/Forschsem/Erdbeeren_yolo/Riseholme/Riseholme-2021/labels/train... 369 images, 0 backgrounds, 0 corrupt: 100%|██████████| 369/369 [00:00<00:00, 755.91it/s]

train: New cache created: /home/parallels/Documents/Forschsem/Erdbeeren_yolo/Riseholme/Riseholme-2021/labels/train.cache


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 246.3±162.0 MB/s, size: 7.8 KB)


val: Scanning /home/parallels/Documents/Forschsem/Erdbeeren_yolo/Riseholme/Riseholme-2021/labels/val... 93 images, 0 backgrounds, 0 corrupt: 100%|██████████| 93/93 [00:00<00:00, 653.25it/s]

val: New cache created: /home/parallels/Documents/Forschsem/Erdbeeren_yolo/Riseholme/Riseholme-2021/labels/val.cache


Plotting labels to runs/train/ripe_only2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/train/ripe_only2
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50         0G     0.8442      2.516      1.399         53        640:  12%|█▎        | 3/24 [01:05<06:52, 19.62s/it]

: 